# Training of MoCoGAN
---
In this Notebook is covered the topic of training the MoCoGAN and the fine tuning of this network.

## Defining global variables and imports
---
In the below cell all of the imports that are needed to the training and some global variables like `device` to use the GPU if available.

In [1]:
import os
import sys
import time
import math
import torch
import skvideo.io
import numpy as np

from glob import glob
from torch.autograd import Variable
from torch import nn, optim, cuda as cu, device, manual_seed

from torch.utils.data import DataLoader
from torchvision.transforms import Lambda, Compose

## Try to avoid problems with dataloader.
"""
import torch.multiprocessing as mp

try:
    mp.set_start_method('forkserver')
    #mp.set_start_method('spawn')
except RuntimeError as errs:
    print(errs)
"""
##########################################

'''Import variables from train.py'''
img_size = 96
nc = 3
ndf = 64 # from dcgan
ngf = 64
d_E = 10
hidden_size = 100 # guess
d_C = 50
d_M = d_E
nz  = d_C + d_M
criterion = nn.BCELoss()

T = 16 # Hyperparameter for taking #Frames into discriminator.

ngpu       = 1
batch_size = 16
n_iter     = 120000
pre_train  = False

## Addition for training on UCF-101
n_epochs_saveV      = 1
n_epochs_display    = 1
n_epochs_check      = 1
max_frame           = 25
cuda                = True #For compatibility with old version of MocoGan
#### End of additions


seed = 0
manual_seed(seed)
np.random.seed(seed)

device = device("cuda" if cu.is_available() else "cpu")

## Import of Models
From the `models` module, let's import all of the models and let's load the previous state for fine tuning.

Then models are moved into the device chosen in the cell above.

In [2]:
sys.path.append("./mocogan/")
from models import Discriminator_I, Discriminator_V, Generator_I ,GRU, UCF_101

In [3]:
'''Create the objects for Discriminator_(I|V), GRU and Generator_I'''
gen_i = Generator_I(nc, ngf, nz, ngpu = 1, batch_size= batch_size)
gru = GRU(d_E, hidden_size, gpu = cu.is_available())
dis_i = Discriminator_I(nc, ndf, ngpu = 1)
dis_v = Discriminator_V(nc, ndf, T = T, ngpu = 1)
gru.initWeight()

'''Move objects into the device chosen'''
''' adjust to cuda '''
if cuda == True:
    dis_i.cuda()
    dis_v.cuda()
    gen_i.cuda()
    gru.cuda()
    criterion.cuda()

'''Optimizer Settings and Optimizer'''
lr = 0.0002
betas=(0.5, 0.999)
optim_Di  = optim.Adam(dis_i.parameters(), lr=lr, betas=betas)
optim_Dv  = optim.Adam(dis_v.parameters(), lr=lr, betas=betas)
optim_Gi  = optim.Adam(gen_i.parameters(), lr=lr, betas=betas)
optim_GRU = optim.Adam(gru.parameters(),   lr=lr, betas=betas)

./mocogan/models.py:238: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(params)
./mocogan/models.py:248: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(params, 0)


## Dataloader
---
In the cells below, the dataloader will be defined and also all of the transformation that will be applied to the videos before taking them into a batch.
To apply transformation to videos, since torchvision does not have a set of methods for this task, a repository called `torch_videovision` has been forked and modified to support transformation on numpy arrays.

In [4]:
from torch_videovision.videotransforms.video_transforms import ColorJitter, RandomTemporalCrop, RandomHorizontalFlip, RandomCrop
from torch_videovision.videotransforms.volume_transforms import ToTensor, ClipToTensor, TransposeChannels
from torch_videovision.videotransforms.tensor_transforms import Normalize, SpatialRandomCrop

/home/carlo/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [5]:
current_path = !pwd
current_path = str(current_path[0])
resized_path = os.path.join(current_path, "mocogan", 'resized_data')
files = glob(os.path.join(resized_path, "*", "*"))

dict_dir = os.path.join(current_path, "mocogan", "ucfTrainTestlist", "classInd.txt")
raw_path = os.path.join(current_path, "mocogan", "raw_data")
raw_files = glob(os.path.join(raw_path, "*", "*"))

'''
transformation = Compose([Lambda(lambda video: video.transpose(3, 0, 1, 2)/255.0),
                            Lambda(lambda video: video[ : , : max_frame, :, : ]),
                            Lambda(lambda video: torch.FloatTensor(video))])
'''

transformation = Compose([  TransposeChannels(),
                            RandomTemporalCrop(max_frame),
                            SpatialRandomCrop( (img_size, img_size) ),
                            RandomHorizontalFlip(),
                            ColorJitter(1, 1, 1, 0.5), #Max Values for Color Jitter
                            ClipToTensor(numpy = True, div_255= True),
                            Normalize(0.5, 0.5),
                            ToTensor()])


dataset = UCF_101(raw_path, dict_dir, supportedExtensions= ["avi"], transform= transformation)
dataloader = DataLoader(dataset, batch_size= batch_size, num_workers= 8, shuffle= True, pin_memory= True, drop_last= True)

## Training
---

In the cells below, first some methods to generate the Noise needed for the GAN is defined, then 

In [6]:
'''Utilities'''
def trim(video):
    start = np.random.randint(0, video.shape[1] - (T+1))
    end = start + T
    return video[:, start:end, :, :]

# for input noises to generate fake video
# note that noises are trimmed randomly from n_frames to T for efficiency
def trim_noise(noise):
    #print("-----TRIMMING NOISE-----")
    #print(f"Noise Size: {noise.size()}")
    start = np.random.randint(0, noise.size(1) - (T+1))
    end = start + T
    #print("-----END OF TRIMMING NOISE-----")
    return noise[:, start:end, :, :, :]


''' calc grad of models '''

def bp_i(inputs, y, retain=False):
    label = (torch.FloatTensor()).cuda()
    label.resize_(inputs.size(0)).fill_(y)
    labelv = Variable(label)
    outputs = dis_i(inputs)
    
    prompt_string = "Fake" if y == 0 else "True"
    print(f"Values from Discriminator_I {outputs}, {prompt_string}  Data")
    
    err = criterion(outputs, labelv)
    err.backward(retain_graph=retain)
    toReturnErr = err.data[0] if err.size() == torch.Tensor().size() else err.item()
    return toReturnErr, outputs.data.mean()

def bp_v(inputs, labels, y, retain=False):
    #print("----BackPropagate_V-----")
    #print(inputs.size())
    label = (torch.FloatTensor()).cuda()
    try:
        label.resize_(inputs.size(0)).fill_(y)

    except RuntimeError as _:
        # Dimension of y does not allow to use fill_
        assert(inputs.size(0) == y.size(0))
        label = (torch.FloatTensor(y)).cuda()

    labelv = Variable(label)
    outputs = dis_v(inputs, labels)
    
    prompt_string = "Fake" if y == 0 else "True"
    print(f"Values from Discriminator_V {outputs}, {prompt_string}  Data")
    
    err = criterion(outputs, labelv)
    err.backward(retain_graph=retain)
    toReturnErr = err.data[0] if err.size() == torch.Tensor().size() else err.item()
    #print("----End of BackPropagate_V-----")
    return toReturnErr, outputs.data.mean()


''' gen input noise for fake video '''

def gen_z(n_frames, batch_size = batch_size):
    #print("----Generating Z-----")
    #print(f"N_FRAMES: {n_frames}")
    #print(f"BATCH_SIZE: {batch_size}")
    #print(f"D_C: {d_C}")
    #print(f"D_E: {d_E}")
    #print(f"nz: {nz}")
    z_C = Variable(torch.randn(batch_size, d_C))
    #  repeat z_C to (batch_size, n_frames, d_C)
    z_C = z_C.unsqueeze(1).repeat(1, n_frames, 1)
    eps = Variable(torch.randn(batch_size, d_E))
    if cuda == True:
        z_C, eps = z_C.cuda(), eps.cuda()

    gru.initHidden(batch_size)
    # notice that 1st dim of gru outputs is seq_len, 2nd is batch_size
    z_M = gru(eps, n_frames).transpose(1, 0)
    z = torch.cat((z_M, z_C), 2)  # z.size() => (batch_size, n_frames, nz)
    #print("----End Generating Z-----")
    return z.view(batch_size, n_frames, nz, 1, 1)

''' prepare for train '''
def timeSince(since):
    now = time.time()
    s = now - since
    d = math.floor(s / ((60**2)*24))
    h = math.floor(s / (60**2)) - d*24
    m = math.floor(s / 60) - h*60 - d*24*60
    s = s - m*60 - h*(60**2) - d*24*(60**2)
    return '%dd %dh %dm %ds' % (d, h, m, s)

trained_path = os.path.join(current_path, "mocogan", 'trained_models')
def checkpoint(model, optimizer, epoch):
    filename = os.path.join(trained_path, '%s_epoch-%d' % (model.__class__.__name__, epoch))
    torch.save(model.state_dict(), filename + '.model')
    torch.save(optimizer.state_dict(), filename + '.state')

def save_video(fake_video, epoch):
    outputdata = fake_video * 255
    outputdata = outputdata.astype(np.uint8)
    dir_path = os.path.join(current_path, 'mocogan', 'generated_videos')
    file_path = os.path.join(dir_path, 'fakeVideo_epoch-%d.mp4' % (epoch))
    skvideo.io.vwrite(file_path, outputdata)


In [7]:
''' train models '''
def train():
    
    start_time = time.time()

    print(f"Starting training: CUDA is { 'On' if cuda == True else 'Off'}")

    for epoch in range(1, n_iter+1):
        ''' prepare real images '''
        # real_videos.size() => (batch_size, nc, T, img_size, img_size)

        # Get data iterator
        updateEvery_Generator_I_GRU = 3
        fake_label = 0
        true_label = [val for val in range(1, 102)]
        data_iter = iter(dataloader) #Iterator
        data_len = len(dataloader) #Num Batches
        data_i = 0

        while data_i < data_len:

            try:
          
                (real_videos, labels) = next(data_iter)
                  
                print(f"\r--------Batch {data_i}/{data_len}---------", end = "")
                #for (key, val) in dataset.class_to_idx.items():
                #    if ( val in labels.tolist() ):
                #        #print(key)
                #        pass

                if cuda == True:
                    real_videos = real_videos.cuda()
                    labels = labels.cuda()
          
                real_videos = Variable(real_videos)
                real_img = real_videos[:, :, np.random.randint(0, T), :, :]

                ''' prepare fake images '''
                # note that n_frames is sampled from video length distribution
                if (len(dataset.videoLengths) > 0):
                    randomVideo = list(dataset.videoLengths)[np.random.randint(0, len(dataset.videoLengths))]
                    n_frames = dataset.videoLengths[randomVideo]
          
                else: # Use this for first iterations, when dataset.videoLengths is not yet updated.
                    n_frames = T + 2 + np.random.randint(0, real_videos.size()[2])
          
                Z = gen_z(n_frames, batch_size)  # Z.size() => (batch_size, n_frames, nz, 1, 1)
                # trim => (batch_size, T, nz, 1, 1)
                Z = trim_noise(Z)
                # generate videos
                Z = Z.contiguous().view(batch_size*T, nz, 1, 1)
                
                fake_videos = gen_i(Z, labels)
                fake_videos = fake_videos.view(batch_size, T, nc, img_size, img_size)
                # transpose => (batch_size, nc, T, img_size, img_size)
                fake_videos = fake_videos.transpose(2, 1)
                # img sampling
                fake_img = fake_videos[:, :, np.random.randint(0, T), :, :]

                ''' train discriminators '''
                # video
                dis_v.zero_grad()
                randomStartFrameIdx = np.random.randint(0, real_videos.size()[2] - T - 1)
                #print("-----INFOS-----")
                #print(f"RandomStartFrame:{randomStartFrameIdx}")
                #print(f"Video Size:{real_videos.size()}")
                #print("-----END OF INFOS-----")
                croppedRealVideos = real_videos[:,:,randomStartFrameIdx: randomStartFrameIdx + T, :, :]
                err_Dv_real, Dv_real_mean = bp_v(croppedRealVideos, labels, 0.9)
                #err_Dv_real, Dv_real_mean = bp_v(croppedRealVideos, labels.type(torch.FloatTensor) / len(dictClassesIdx))
                err_Dv_fake, Dv_fake_mean = bp_v(fake_videos.detach(), labels, fake_label)
                err_Dv = err_Dv_real + err_Dv_fake
                optim_Dv.step()
                # image
                dis_i.zero_grad()
                err_Di_real, Di_real_mean = bp_i(real_img, 0.9)
                err_Di_fake, Di_fake_mean = bp_i(fake_img.detach(), fake_label)
                err_Di = err_Di_real + err_Di_fake
                optim_Di.step()


                ''' train generators '''
                gen_i.zero_grad()
                gru.zero_grad()
                # video. notice retain=True for back prop twice
                err_Gv, _ = bp_v(fake_videos, labels, 0.9, retain=True)
                # images
                err_Gi, _ = bp_i(fake_img, 0.9)
          
                if epoch % updateEvery_Generator_I_GRU == 0:
                    optim_Gi.step()
                    optim_GRU.step()

                '''Increment index for Batch'''
                data_i = data_i + 1
          
                '''Cool down the Hardware'''
                time.sleep(1/2)
          
            except StopIteration:
                break
            
            except KeyboardInterrupt:
                save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)
                checkpoint(dis_i, optim_Di, epoch)
                checkpoint(dis_v, optim_Dv, epoch)
                checkpoint(gen_i, optim_Gi, epoch)
                checkpoint(gru,   optim_GRU, epoch)

        if epoch % n_epochs_display == 0:
            print('[%d/%d] (%s) Loss_Di: %.4f Loss_Dv: %.4f Loss_Gi: %.4f Loss_Gv: %.4f Di_real_mean %.4f Di_fake_mean %.4f Dv_real_mean %.4f Dv_fake_mean %.4f'
                  % (epoch, n_iter, timeSince(start_time), err_Di, err_Dv, err_Gi, err_Gv, Di_real_mean, Di_fake_mean, Dv_real_mean, Dv_fake_mean))

        if epoch % n_epochs_saveV == 0:
            save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)

        if epoch % n_epochs_check == 0:
            checkpoint(dis_i, optim_Di, epoch)
            checkpoint(dis_v, optim_Dv, epoch)
            checkpoint(gen_i, optim_Gi, epoch)
            checkpoint(gru,   optim_GRU, epoch)
          
        '''Cool down the Hardware'''
        time.sleep(5)
          

## Loading Previous State
---
If wanted, the following cell can be used to load the previous state of a trained model.

In [8]:
''' use pre-trained models '''

def load():
    dis_i.load_state_dict(torch.load(trained_path + '/Discriminator_I.model'))
    dis_v.load_state_dict(torch.load(trained_path + '/Discriminator_V.model'))
    gen_i.load_state_dict(torch.load(trained_path + '/Generator_I.model'))
    gru.load_state_dict(torch.load(trained_path + '/GRU.model'))
    optim_Di.load_state_dict(torch.load(trained_path + '/Discriminator_I.state'))
    optim_Dv.load_state_dict(torch.load(trained_path + '/Discriminator_V.state'))
    optim_Gi.load_state_dict(torch.load(trained_path + '/Generator_I.state'))
    optim_GRU.load_state_dict(torch.load(trained_path + '/GRU.state'))

## Finally, start training


In [9]:
#load()
train() 

Starting training: CUDA is On
(3, 25, 96, 96) <class 'numpy.ndarray'> [[[[213 206 214 ... 144 150 151]
   [255 255 255 ... 148 155 155]
   [241 237 234 ... 152 158 158]
   ...
   [243 240 237 ...  43  41  41]
   [243 240 237 ...  43  41  41]
   [243 240 238 ...  43  41  41]]

  [[183 177 184 ... 150 151 154]
   [240 241 247 ... 155 155 158]
   [242 241 241 ... 158 158 160]
   ...
   [241 239 236 ...  43  41  41]
   [241 239 236 ...  43  41  41]
   [241 239 237 ...  43  41  41]]

  [[174 190 196 ... 144 147 152]
   [234 252 254 ... 153 154 158]
   [238 244 241 ... 157 158 160]
   ...
   [251 246 242 ...  43  41  41]
   [250 245 241 ...  43  41  41]
   [246 244 241 ...  43  41  41]]

  ...

  [[ 62  63  66 ...  73  74  74]
   [ 93  93  98 ...  79  78  75]
   [214 214 222 ... 100  95  90]
   ...
   [ 63  62  67 ...  44  44  44]
   [ 49  54  58 ...  44  44  44]
   [ 66  75  73 ...  44  44  44]]

  [[ 62  63  66 ...  73  74  74]
   [ 93  93  98 ...  79  78  75]
   [214 214 222 ... 100  95  

   [138 138 138 ... 147 147 146]]]]
(3, 25, 96, 96) <class 'numpy.ndarray'> [[[[ 75  75  75 ... 249 249 249]
   [ 75  75  76 ... 250 250 250]
   [ 75  75  76 ... 252 252 252]
   ...
   [ 78  95 102 ...  47  45  45]
   [ 81  93  93 ...  47  45  45]
   [ 85  92  87 ...  47  45  45]]

  [[ 75  75  75 ... 249 249 249]
   [ 75  75  76 ... 250 250 250]
   [ 75  75  76 ... 252 252 252]
   ...
   [ 78  95 102 ...  48  46  46]
   [ 81  93  93 ...  48  46  46]
   [ 85  92  87 ...  48  46  46]]

  [[ 78  78  77 ... 248 248 248]
   [ 78  78  77 ... 250 250 250]
   [ 78  78  80 ... 252 252 252]
   ...
   [ 76  92  93 ...  49  47  47]
   [ 80  95  85 ...  49  47  47]
   [ 93  83  48 ...  49  47  47]]

  ...

  [[ 85  85  87 ... 230 249 254]
   [ 84  85  85 ... 238 252 253]
   [ 84  84  85 ... 247 254 254]
   ...
   [103 130 122 ...  54  48  43]
   [108 129 112 ...  55  47  40]
   [118 125 104 ...  54  45  38]]

  [[ 85  85  84 ... 234 251 253]
   [ 84  85  84 ... 244 254 255]
   [ 84  84  87 ... 251

AttributeError: Traceback (most recent call last):
  File "/home/carlo/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/carlo/anaconda3/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "./mocogan/models.py", line 441, in __getitem__
    readVideo = self.transform(readVideo)
  File "/home/carlo/anaconda3/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/home/carlo/Documenti/Text2VideoGAN/torch_videovision/videotransforms/tensor_transforms.py", line 30, in __call__
    return F.normalize(tensor, self.mean, self.std)
  File "/home/carlo/Documenti/Text2VideoGAN/torch_videovision/videotransforms/utils/functional.py", line 9, in normalize
    tensor.sub_(mean).div_(std)
AttributeError: 'numpy.ndarray

(3, 25, 96, 96) <class 'numpy.ndarray'> [[[[ 42  42  42 ...  40  40  40]
   [ 42  42  42 ...  40  40  40]
   [ 42  42  42 ...  40  40  40]
   ...
   [  2   1   1 ...   0   0   0]
   [ 11  10   9 ...   2   3   3]
   [  4   3   3 ...   0   0   0]]

  [[ 45  45  45 ...  41  38  37]
   [ 46  46  46 ...  41  39  38]
   [ 46  46  46 ...  41  40  39]
   ...
   [  2   1   1 ...   1   2   2]
   [ 11  10   8 ...   2   1   0]
   [  4   3   2 ...   0   0   4]]

  [[ 41  41  41 ...  95  80  58]
   [ 42  42  42 ... 100  86  71]
   [ 45  45  45 ...  97  90  83]
   ...
   [  3   2   2 ...   1   2   0]
   [  8   6   5 ...   1   0   0]
   [  4   3   2 ...   0   1   6]]

  ...

  [[ 41  41  41 ...  41  41  40]
   [ 41  41  41 ...  43  41  41]
   [ 42  42  42 ...  43  41  41]
   ...
   [  1   1   1 ...   0   0   0]
   [  0   0   0 ...   0   0   0]
   [  0   0   0 ...   1   1   1]]

  [[ 41  41  41 ... 112  76  38]
   [ 42  42  42 ... 112  79  39]
   [ 42  42  42 ... 114  81  41]
   ...
   [  1   1   1 ...